<a href="https://colab.research.google.com/github/HassanDataSci/caricon-personality-career-matching/blob/main/CariCon_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load each file into its own DataFrame
df_mbti   = pd.read_excel("myers-briggs-data.xlsx", engine="openpyxl")
df_chars  = pd.read_excel("onet_data_with_human_characteristics.xlsx", engine="openpyxl")
df_salary = pd.read_excel("onet_data_with_salaries.xlsx", engine="openpyxl")

print("MBTI shape:", df_mbti.shape)
print("Human characteristics shape:", df_chars.shape)
print("Salaries shape:", df_salary.shape)

MBTI shape: (8675, 2)
Human characteristics shape: (1016, 30)
Salaries shape: (1016, 29)


In [3]:
print("=== MBTI Data ===")
print(df_mbti.dtypes)
print()

print("=== O*NET Human Characteristics ===")
print(df_chars.dtypes)
print()

print("=== O*NET Salaries ===")
print(df_salary.dtypes)

=== MBTI Data ===
type     object
posts    object
dtype: object

=== O*NET Human Characteristics ===
Occupation Code              object
Title                        object
Description                  object
Sample Job Titles            object
Tasks                        object
Knowledge                    object
Skills                       object
Abilities                    object
Work Activities              object
Detailed Work Activities     object
Tools Used                   object
Technology Used              object
Job Zone                     object
Education Level              object
Experience Required          object
Job Training                 object
SVP Range                    object
Job Zone Examples            object
Interests                    object
Work Styles                  object
Work Values                  object
Work Context                 object
Additional Sources           object
Related Occupations          object
Annual 10th Percentile      float64